In [117]:
import pyspark 
import random
from pyspark.sql import SQLContext, functions as func, types as typ
from pyspark.sql.functions  import explode, split, avg, col, desc, asc

sc = pyspark.SparkContext.getOrCreate() 
## Read the data into memory

sqlContest = SQLContext(sc)

mv1 = sqlContest.read.csv('ml-latest-small/movies.csv', header=True)
movies = sqlContest.read.csv('ml-latest-small/movies.csv', header=True)
links = sqlContest.read.csv('ml-latest-small/links.csv', header=True)
ratings = sqlContest.read.csv('ml-latest-small/ratings.csv', header=True)
tags = sqlContest.read.csv('ml-latest-small/tags.csv', header=True)

#print(mv1.first())
##Convert the timestamp to date

ratings = ratings.withColumn('RatingDate', func.date_format(ratings.timestamp.
                                                       cast(dataType= typ.LongType()).
                                                       cast(dataType= typ.TimestampType()),
                                                       "yyyy-MM-dd"))
ratings = ratings.withColumn("rating", ratings.rating.cast(dataType = typ.FloatType()))
tags = tags.withColumn('tagDate', func.date_format(tags.timestamp.
                                                       cast(dataType= typ.LongType()).
                                                       cast(dataType= typ.TimestampType()),
                                                   "yyyy-MM-dd"))

#Split the genres column into rows based on the number of the separator | 

moviesGeners =movies.withColumn("Genr",explode(split("genres","[|]")))

#moviesGeners.show()
#df2 = moviesGeners.join(ratings, on=['movieId'], how='inner')
#df2.groupBy("Genr").count().show()
#moviesGeners.select("Genres").distinct().show()

#print("Number of movies in all genres = " + str(df.count()))

## Add geners columns using binary encoding for each category
#genersLabels = ["Action", "Adventure", "Animation", "Children's", "Comedy"
#                "Crime", "Documentary","Drama", "Fantasy", "Film-Noir",
#                "Horror", "Musical", "Mystery", "Romance", "Sci-Fi",
#                "Thriller", "War", "Western"]

#movies1 = movies
#for label in genersLabels:
#    movies = movies.withColumn(label, movies["genres"].like("%" + label +"%"))

#geners = df.select("genres").distinct()

#geners.show()
#print(geners.count())
##Persist the data in memory
movies.persist()
links.persist()
ratings.persist()
tags.persist()

#ratings.printSchema()
#Covert timestamp seconds to date

def cleanTheDataSet():
    
    return

#print(ratings.count())
## Basic Requirements

def searchUserById(id_):
    filtered = ratings.where("userId =" + str(id_))
    filtered = filtered.join(movies, on=['movieId'], how ='inner')
    
    return filtered

def searchUsersById(ids):
    
    return 

def searchMovieById(id):
    filteredMovies = movies.where("movieId = " + str(id))
    
    return filteredMovies

def searchMovieByTitle(title):
    filteredMovies = movies.where("title like \'%" + str(title) + "%\'")
    filteredMovies.show()
    return 

def searchGenre(gen):
    filteredMovies = movies.where("genres like \'%" + str(gen) + "%\'")
    #filteredMovies.show()
    return filteredMovies

def searchGenres(gens):
       
        
    return

def searchMoviesByYear(year):
    filteredMovies = ratings.where("RatingDate like \'" + str(year) + "%\'")
    filteredMovies.show()
    return

def listTopNRated(n):
    top_rated = ratings\
    .groupBy("movieId")\
    .agg(avg(col("rating")))\
    .withColumnRenamed("avg(rating)", "Average_Rating")\
    .join(movies, on=['movieId'], how='inner')\
    .sort(desc("Average_Rating"))
    print("Top Rated")
    top_rated.show(n)
    return


def listTopNWatched(n):
    most_popular = ratings\
    .groupBy("movieId")\
    .agg(func.count("userId"))\
    .withColumnRenamed("count(userId)", "Number_OF_Rating")\
    .sort(func.desc("Number_OF_Rating"))
    most_popular = most_popular.join(movies, on=['movieId'], how='inner')
    most_popular.show(n)
    return


## Intermediate Requirements:

def findUsersGenre(userIds):
    
    userMovies = searchUserById(userIds)
    userGenres = userMovies.join(moviesGeners, on=['movieId'], how='inner')
    userGenres = userGenres.groupBy("Genr")\
    .agg(avg(col("rating")), func.count(col("userId")))\
    .withColumnRenamed("avg(rating)", "Average_Rating")\
    .withColumnRenamed("count(userId)", "wachings")\
    .sort(desc("Average_Rating"))#.where("Average_Rating >=3 ")\
    
    return userGenres

def findUsersFavourite(userId):
    print("Favourite Geners of User:" + str(userId))
    userFav = findUsersGenre(userId).where("Average_Rating > 3.0")
    
    return userFav
    
def compareTastes(userId1, userId2):
    u1 = findUsersGenre(userId1)
    u2 = findUsersGenre(userId2)
    u1 = u1.withColumnRenamed("wachings","User_" + str(userId1) +"_Watching_Times")\
    .withColumnRenamed("Average_Rating","User_" + str(userId1) +"_Rating")
    u2 = u2.withColumnRenamed("wachings","User_" + str(userId2) +"_Watching_Times")\
    .withColumnRenamed("Average_Rating","User_" + str(userId2) +"_Rating")    
    u1 = u1.join(u2, on=['Genr'], how='outer')
    print("Comparing the Tastes of user:" +
         str(userId1) + " , and user:" + str(userId2) + " :" )
    
    return u1

## Advanced Requirements:

def clusterUsersByTaste():
    
    return 0

def visualizeTheDataSet():
    
    return 0

def recommendMovie(userId):
    
    return 0

compareTastes(1,2).show()
findUsersFavourite(2).show()
#searchMoviesByYear(2016)
#listTopNRated(20)
#searchMovieById(2)
#searchGenre("Action")
#searchMovieByTitle("Jum")
#searchUserById(5)
#searchUserById(22)
#listTopNWatched(5)

Comparing the Tastes of user:1 , and user:2 :
+-----------+------------------+---------------------+------------------+---------------------+
|       Genr|     User_1_Rating|User_1_Watching_Times|     User_2_Rating|User_2_Watching_Times|
+-----------+------------------+---------------------+------------------+---------------------+
|      Crime| 4.355555555555555|                   45|               3.8|                   10|
|    Romance|4.3076923076923075|                   26|               4.5|                    1|
|   Thriller|4.1454545454545455|                   55|               3.7|                   10|
|  Adventure|4.3882352941176475|                   85| 4.166666666666667|                    3|
|      Drama| 4.529411764705882|                   68|3.8823529411764706|                   17|
|        War|               4.5|                   22|               4.5|                    1|
|Documentary|              null|                 null| 4.333333333333333|                 